In [2]:
from fastai.text.all import *
from fastai.callback.progress import CSVLogger

In [7]:
model_path = Path('data/model')
path = Path('data/dewiki_160k/docs_160k')
lang = 'de'

In [3]:
bs=128

In [4]:
tok = SentencePieceTokenizer(lang=lang)

In [5]:
dblock = DataBlock(blocks=TextBlock.from_folder(path, is_lm=True, tok=tok),
                   get_items=get_files,
                   splitter=RandomSplitter(valid_pct=0.1, seed=42),
                  )

dls = dblock.dataloaders(path, path=path, bs=bs, num_workers=18)

In [6]:
dls.show_batch()

,text,text_
0,"▁xxbos ▁xxup ▁us ▁xxmaj ▁ men ’ s ▁xxmaj ▁clay ▁xxmaj ▁court ▁xxmaj ▁championship s ▁2009 ▁xxmaj ▁die ▁xxup ▁us ▁xxmaj ▁ men ’ s ▁xxmaj ▁clay ▁xxmaj ▁court ▁xxmaj ▁championship s ▁2009 ▁waren ▁ein ▁xxmaj ▁tennis turnier , ▁welches ▁vom ▁4. ▁bis ▁12. ▁xxmaj ▁april ▁2009 ▁in ▁xxmaj ▁houston ▁stattfand . ▁xxmaj ▁es ▁war ▁xxmaj ▁teil ▁der ▁xxup ▁atp ▁xxmaj ▁world ▁xxmaj ▁tour ▁2009 ▁und ▁wurde ▁im ▁xxmaj ▁freien ▁auf ▁xxmaj","▁xxup ▁us ▁xxmaj ▁ men ’ s ▁xxmaj ▁clay ▁xxmaj ▁court ▁xxmaj ▁championship s ▁2009 ▁xxmaj ▁die ▁xxup ▁us ▁xxmaj ▁ men ’ s ▁xxmaj ▁clay ▁xxmaj ▁court ▁xxmaj ▁championship s ▁2009 ▁waren ▁ein ▁xxmaj ▁tennis turnier , ▁welches ▁vom ▁4. ▁bis ▁12. ▁xxmaj ▁april ▁2009 ▁in ▁xxmaj ▁houston ▁stattfand . ▁xxmaj ▁es ▁war ▁xxmaj ▁teil ▁der ▁xxup ▁atp ▁xxmaj ▁world ▁xxmaj ▁tour ▁2009 ▁und ▁wurde ▁im ▁xxmaj ▁freien ▁auf ▁xxmaj ▁sand"
1,"▁xxmaj ▁saarlandes ▁zur ▁xxmaj ▁bundesrepublik ▁xxmaj ▁deutschland ▁das ▁xxmaj ▁unterscheidung s zeichen ▁"" ig b "" ▁zugewiesen . ▁xxmaj ▁es ▁wurde ▁bis ▁zum ▁28. ▁xxmaj ▁februar ▁1974 ▁ausgegeben . ▁xxmaj ▁für ▁die ▁xxmaj ▁dauer ▁eines ▁xxmaj ▁monats ▁war ▁es ▁nicht ▁erhältlich . ▁xxmaj ▁seit ▁dem ▁1. ▁xxmaj ▁april ▁1974 ▁wird ▁es ▁in ▁der ▁xxmaj ▁mittel stadt ▁xxmaj ▁sankt ▁xxmaj ▁ ing bert , ▁die ▁heute ▁zum ▁saar pfalz - kreis ▁gehört ,","▁saarlandes ▁zur ▁xxmaj ▁bundesrepublik ▁xxmaj ▁deutschland ▁das ▁xxmaj ▁unterscheidung s zeichen ▁"" ig b "" ▁zugewiesen . ▁xxmaj ▁es ▁wurde ▁bis ▁zum ▁28. ▁xxmaj ▁februar ▁1974 ▁ausgegeben . ▁xxmaj ▁für ▁die ▁xxmaj ▁dauer ▁eines ▁xxmaj ▁monats ▁war ▁es ▁nicht ▁erhältlich . ▁xxmaj ▁seit ▁dem ▁1. ▁xxmaj ▁april ▁1974 ▁wird ▁es ▁in ▁der ▁xxmaj ▁mittel stadt ▁xxmaj ▁sankt ▁xxmaj ▁ ing bert , ▁die ▁heute ▁zum ▁saar pfalz - kreis ▁gehört , ▁ausgegeben"
2,"▁und ▁umstritten e ▁xxmaj ▁journalist ▁xxmaj ▁maximilian ▁xxmaj ▁hard en ▁publizierte ▁in ▁der ▁von ▁ihm ▁herausgegeben en ▁xxmaj ▁zeitschrift ▁"" die ▁xxmaj ▁zukunft "" ▁im ▁xxmaj ▁sommer ▁1910 ▁nacheinander ▁drei ▁xxmaj ▁artikel ▁zum ▁xxmaj ▁fall , ▁in ▁denen ▁er ▁sehr ▁detailliert ▁aus ▁sexual pathologisch er ▁xxmaj ▁sicht ▁auf ▁den ▁xxmaj ▁täter ▁xxmaj ▁hugo ▁von ▁xxmaj ▁go e ben ▁ein ging . ▁xxmaj ▁sein ▁xxmaj ▁ kollege ▁xxmaj ▁karl ▁xxmaj ▁kraus ▁attackiert e","▁umstritten e ▁xxmaj ▁journalist ▁xxmaj ▁maximilian ▁xxmaj ▁hard en ▁publizierte ▁in ▁der ▁von ▁ihm ▁herausgegeben en ▁xxmaj ▁zeitschrift ▁"" die ▁xxmaj ▁zukunft "" ▁im ▁xxmaj ▁sommer ▁1910 ▁nacheinander ▁drei ▁xxmaj ▁artikel ▁zum ▁xxmaj ▁fall , ▁in ▁denen ▁er ▁sehr ▁detailliert ▁aus ▁sexual pathologisch er ▁xxmaj ▁sicht ▁auf ▁den ▁xxmaj ▁täter ▁xxmaj ▁hugo ▁von ▁xxmaj ▁go e ben ▁ein ging . ▁xxmaj ▁sein ▁xxmaj ▁ kollege ▁xxmaj ▁karl ▁xxmaj ▁kraus ▁attackiert e ▁ihn"
3,"▁„ großen ▁xxmaj ▁ göttin “ ▁geleitet ▁gewesen ▁und ▁anschließend ▁von ▁ patri linear en ▁xxmaj ▁hor den ▁ kulturell ▁über fremd et ▁worden ▁sei , ▁wurden ▁in ▁ihrer ▁archäologischen ▁xxmaj ▁daten basis ▁und ▁ihrer ▁erkenntnis theoretischen ▁xxmaj ▁grundlage ▁in ▁xxmaj ▁frage ▁gestellt . ▁xxmaj ▁der ▁britische ▁xxmaj ▁ archäologe ▁xxmaj ▁colin ▁xxmaj ▁ ren fre w ▁vertritt ▁dagegen ▁in ▁seiner ▁sogenannten ▁ anatolien - hypothese , ▁dass ▁sich ▁die ▁xxmaj ▁","großen ▁xxmaj ▁ göttin “ ▁geleitet ▁gewesen ▁und ▁anschließend ▁von ▁ patri linear en ▁xxmaj ▁hor den ▁ kulturell ▁über fremd et ▁worden ▁sei , ▁wurden ▁in ▁ihrer ▁archäologischen ▁xxmaj ▁daten basis ▁und ▁ihrer ▁erkenntnis theoretischen ▁xxmaj ▁grundlage ▁in ▁xxmaj ▁frage ▁gestellt . ▁xxmaj ▁der ▁britische ▁xxmaj ▁ archäologe ▁xxmaj ▁colin ▁xxmaj ▁ ren fre w ▁vertritt ▁dagegen ▁in ▁seiner ▁sogenannten ▁ anatolien - hypothese , ▁dass ▁sich ▁die ▁xxmaj ▁ indo"
4,"▁im ▁hessischen ▁xxmaj ▁amateur fußball ▁fort ▁und ▁war ▁in ▁dieser ▁xxmaj ▁funktion ▁beim ▁xxup ▁sv ▁xxmaj ▁kri f tel ▁(19 78) , ▁der ▁xxup ▁ sg ▁xxmaj ▁ kel k heim ▁(19 80 –19 82) ▁und ▁der ▁spvgg ▁xxmaj ▁ selig en stadt ▁(19 82) ▁aktiv . ▁xxbos ▁xxmaj ▁james ▁xxup ▁ o . ▁xxmaj ▁ pag e ▁xxmaj ▁james ▁xxup ▁ o . ▁xxmaj ▁ pag e ▁(* ▁7. ▁xxmaj ▁august"

In [7]:
len(dls.train), len(dls.valid)

(20281, 2253)

In [8]:
learn = language_model_learner(dls, AWD_LSTM, drop_mult=0.5, pretrained=False, 
                               metrics=[accuracy, Perplexity()]).to_native_fp16()
learn.path = model_path

In [9]:
lr = 1e-2
lr *= bs/48  # Scale learning rate by batch size

In [10]:
cbs=[CSVLogger(fname='history_de_wikitext.csv')]

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, cbs=cbs, moms=(0.8, 0.7, 0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time


In [ ]:
lm_fns = [model_path/f'{lang}_wikitext', model_path/f'{lang}_wikitext_vocab.pkl']

In [ ]:
learn.to_fp32().save(lm_fns[0], with_opt=False)

In [ ]:
with open(lm_fns[1], 'wb') as f:
      pickle.dump(learn.dls.vocab, f)